# Assignment of peer feedback

This notebook contains the code used to determine which two other teams each team must give feed back to in the data project

In [ ]:
!pip install PyGithub

In [5]:
from github import Github 
import sys
import datetime
import pandas as pd
import numpy as np
import random
from collections import Counter

In [77]:
# Which project are we finding peer reviewing for?
all_projects= ['inauguralproject', 'dataproject', 'modelproject']
project = 'inauguralproject' 
seed = all_projects.index(project)
year = "2022"   

In [22]:
# Load personal access token
with open("/Users/jzk870/Documents/NumericalMethods - Harddisk/token/git_access_token.txt", mode = "r") as file:
    token = file.read()

In [45]:
# a. Access github through access token. This one has been destroyed by Github for safety reasons.
gh = Github(token)
org = gh.get_organization('NumEconCopenhagen')
all_repos = org.get_repos()

# b. Locate all repositories in current class 
class_name = "projects-" + year
current_class = [repo.name for repo in all_repos if class_name in repo.name]

In [75]:
# c. Structures  
active_repos = {}
date0 = datetime.datetime(2022, 3, 1)
handin_date = datetime.datetime(2022, 5, 13)
disregard_repos = ["projects-2022-slet_mig_senere","projects-2022-crashtest"]

In [47]:
# Function to test whether activity has been recorded since active date in specific project
def check_activity(repo, project, active_repos, start_date, handin_date):
    try:
        commits = repo.get_commits(since = start_date, until = handin_date)
        for commit in commits:
            for cf in commit.files:
                if project in cf.filename:
                    active_repos[repo.name] = repo
                    return active_repos
    except:
        pass    
    return active_repos

In [61]:
# Add active repos to dictionary
i = 1
for repo in all_repos:
    
    if repo.name not in current_class:
        continue
    
    if repo.name in disregard_repos:
        continue 
        
    print('processing repo', str(i), 'out of', str(len(current_class)), end = '\r')
    
    # Run function to test presence an active data project
    active_repos = check_activity(repo, project, active_repos, date0, handin_date)
    i += 1

In [ ]:
# Get team names and write out all active repos
teams = []
for k in active_repos.keys():
    teams.append(k.replace(class_name+'-', ''))
teams.sort()

In [78]:
# Create data frame for storing assignment
random.seed(seed)
teams_df = pd.DataFrame(data = teams, columns = ['team'])
irrelevant = teams_df.team.isin(disregard_repos)
teams_df = teams_df.loc[np.invert(irrelevant),:]
teams_df.sort_values(by = 'team', inplace = True)
teams_df.reset_index(drop=True, inplace=True)
teams_df['peer_group_1'] = np.nan
teams_df['peer_group_2'] = np.nan
teams_df.head()

,team,peer_group_1,peer_group_2
0,2-brown-1-white,NaN,NaN
1,21,NaN,NaN
2,2h2h,NaN,NaN
3,alberte-og-eva,NaN,NaN
4,amanda-oliver,NaN,NaN


In [79]:
# Algorithm for assigning peers to teams
N = len(teams_df.team.values)
counter = [0 for _ in range(N)]
assigned = []

# For each team, loop over teams that have not already been assigned
for i,_ in enumerate(teams_df.team):
    pop = [t for t in range(N) if (t != i) & (t not in assigned)]
    peers = random.sample(pop, 2)    
    for c in [0,1]:
        teams_df.iloc[i, 1 + c] = teams_df.team[peers[c]]
        counter[peers[c]] += 1
        if counter[peers[c]] == 2:
            assigned.append(peers[c])

# see teams_df in variable inspector 

In [ ]:
# Test that everyone are assigned to exactly 2 groups
peergroups = teams_df.peer_group_1.values.tolist() + teams_df.peer_group_2.values.tolist()
counts = Counter(peergroups)
print('Everyone is assigned to exactly 2 groups?: ', set(counts.values()) == {2})

Everyone is assigned to exactly 2 groups?:  True


Store the resulting list in excel file available on MS Teams

In [86]:
teams_df.to_excel('Model Project - peer assignment.xlsx', sheet_name=project, index = False)